End to end machine learning workflow to predict mortality within 180 days of last visit


This is step 3

We will populate the Charlson Comorbility Index score for each patient in our training set

# Importing Packages and Data

In [2]:
# Package Import

import pandas as pd

In [10]:
# import data

demographics = pd.read_csv('output/step_2_train_demographics')
conditions = pd.read_csv('input/condition_occurrence.csv')

/var/folders/12/24w9w2w944x4_v3m36w_mnth0000gn/T/ipykernel_59260/2205000204.py:4: DtypeWarning: Columns (2,3,7,8,13) have mixed types. Specify dtype option on import or set low_memory=False.
  conditions = pd.read_csv('input/condition_occurrence.csv')


# Calculate Charlson Comorbility Index

In [11]:
# Preview demographics

demographics.head()

,person_id,year_of_birth,gender_source_value,ethnicity_source_value,death_datetime,latest_visit_date,age_last_visit,outcome
0,138730,1981.0,1,1,2009-10-02 11:50:35,2009-07-07 13:12:14,28.0,1
1,127727,1944.0,1,2,2009-07-25 23:38:38,2009-02-21 13:11:27,65.0,1
2,46210,1941.0,1,3,2010-08-03 19:47:22,2010-05-20 12:20:52,69.0,1
3,10249,1929.0,2,2,2010-05-26 23:14:58,2009-12-01 16:23:09,80.0,1
4,78014,1931.0,1,1,2010-07-23 23:37:41,2010-05-05 05:51:12,79.0,1


In [12]:
# Preview conditions

conditions.head()

,provider_id,visit_detail_id,condition_start_datetime,condition_end_datetime,condition_source_concept_id,person_id,condition_status_source_value,condition_end_date,condition_start_date,condition_status_concept_id,condition_type_concept_id,condition_concept_id,stop_reason,condition_source_value,condition_occurrence_id,visit_occurrence_id
0,NaN,NaN,2008-11-20,NaN,44829639.0,115208.0,NaN,NaN,2008-11-20,4230359.0,38000230.0,312922.0,NaN,V4581,9070047.0,NaN
1,NaN,NaN,2009-02-18,NaN,44825447.0,115208.0,NaN,NaN,2009-02-18,4230359.0,38000230.0,43020432.0,NaN,44021,5258637.0,NaN
2,NaN,NaN,2008-08-24,NaN,44828074.0,115208.0,NaN,NaN,2008-08-24,NaN,38000230.0,75354.0,NaN,7329,12839271.0,NaN
3,NaN,NaN,2010-05-14,NaN,44830080.0,115208.0,NaN,NaN,2010-05-14,4230359.0,38000230.0,42872402.0,NaN,41401,4019921.0,2042432.0
4,NaN,NaN,2009-02-08,NaN,44828981.0,115208.0,NaN,NaN,2009-02-08,4230359.0,38000230.0,316999.0,NaN,4279,4022174.0,NaN


In [13]:
# Filter conditions to only include the features we need
# reset index

conditions = conditions[['person_id','condition_concept_id']]
conditions = conditions.reset_index(drop=True)

In [14]:
# Create a list of the person id's from out training set

patients = list(demographics['person_id'])

In [15]:
# Store Charlson index concept id's and the weight

condition_concept_id = [312337.0,319835.0,321052.0,441002.0,4182210.0,439276.0,4043378.0,
                        374888.0,375791.0,80809.0,441928.0,257628.0,80800.0,192680.0,200763.0,
                        201254.0,201826.0,374022.0,443611.0,443597.0,443612.0,317510.0,432571.0,433740.0]

concept_weight = [1,1,1,1,1,1,1,1,1,1,1,1,1,3,1,1,1,2,2,2,2,2,2,1]

In [16]:
# filter the conditions table to only include the concept id's we need and the patients id from training set

conditions = conditions.loc[conditions.condition_concept_id.isin(condition_concept_id)]
conditions = conditions.reset_index(drop=True)

conditions = conditions.loc[conditions.person_id.isin(patients)]
conditions = conditions.reset_index(drop=True)

In [18]:
# Create a list containing a string version of each concept id to use as column names

condition_concept_id_string = []
for i in condition_concept_id:
    condition_concept_id_string.append(str(i))

In [23]:
# Create a new data frame and initialize each value to 0
# The rows will be unique patients
# the columns will be the concept codes from the charlson index


num_person = len(patients) #rows
num_condition_concept_id = len(condition_concept_id) #columns


populated_data = []
for i in range(len(patients)):
    current_row = []
    for j in range(num_condition_concept_id):
        current_row.append(0)
    populated_data.append(current_row)
    
    
    
# Convert the 2 dimensional list to a pandas df
# Set the columns to be the list of string concept id's
populated_data = pd.DataFrame(populated_data,columns=condition_concept_id_string)

# Insert the patient id's as the first column so we can reference each patient later
populated_data.insert(loc=0, column='person_id', value=patients)

In [24]:
# Preview our created df initialized to each condition being 0

populated_data.head()

,person_id,312337.0,319835.0,321052.0,441002.0,4182210.0,439276.0,4043378.0,374888.0,375791.0,...,200763.0,201254.0,201826.0,374022.0,443611.0,443597.0,443612.0,317510.0,432571.0,433740.0
0,138730,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,127727,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,46210,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10249,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,78014,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
# We Populate our df by going through each row in conditions 
# Conditions has two columns: person_id and condition_concept_id
# We store the current person and the current concept_id
# We then loop through the patients list and find the index of the person we want
# Then we just reference the df location by matching the person index and string of concept id


for i in range(len(conditions)):
    person = conditions.loc[i][0]
    concept_id = conditions.loc[i][1]

    for j in range(len(patients)):
        if(patients[j] == person):
            person_index = j
    populated_data.loc[person_index][str(concept_id)] = 1

In [26]:
# Preview our populated df

populated_data.head()

,person_id,312337.0,319835.0,321052.0,441002.0,4182210.0,439276.0,4043378.0,374888.0,375791.0,...,200763.0,201254.0,201826.0,374022.0,443611.0,443597.0,443612.0,317510.0,432571.0,433740.0
0,138730,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
1,127727,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,1,0,0,0
2,46210,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10249,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,78014,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# Since several Charlson Index features have multiple concept codes we need to go through and check
# each concept code and populate either the weight, or if it has multple, select the max value
# from each category. For example: a patient can have several dimensia codes so the max will just be 1
# we then store that into a list 


weighted_charlson = []
for i in range(len(populated_data)):
    s = 0 
    #Mi
    if(populated_data.loc[i]['312337.0']==1):
        s = s+1
    #CHF
    if(populated_data.loc[i]['319835.0']==1):
        s = s+1
    # PVD
    if(populated_data.loc[i]['321052.0']==1):
        s = s+1
    #Dementia
    s = s + max(populated_data.loc[i]['441002.0'],populated_data.loc[i]['4182210.0'],populated_data.loc[i]['439276.0'],populated_data.loc[i]['4043378.0'],populated_data.loc[i]['374888.0'],populated_data.loc[i]['375791.0'])
    #Connective tissue disease
    s = s + max(populated_data.loc[i]['80809.0'],populated_data.loc[i]['441928.0'],populated_data.loc[i]['257628.0'],populated_data.loc[i]['80800.0'])
    # Liver disease
    if(populated_data.loc[i]['192680.0']==1):
        s = s+3
    if(populated_data.loc[i]['200763.0']==1):
        s = s+1
    #Diabetes Mellitus
    if(populated_data.loc[i]['201254.0']==1):
        s = s+1
    if(populated_data.loc[i]['201826.0']==1):
        s = s+1
    #Hemiplegia
    if(populated_data.loc[i]['374022.0']==1):
        s = s+2
    # Chronic kidney disease 3-5
    s = s + 2*max(populated_data.loc[i]['443611.0'],populated_data.loc[i]['443597.0'],populated_data.loc[i]['443612.0'])
    # Leukemia
    if(populated_data.loc[i]['317510.0']==1):
        s = s+2
    #Lymphoma
    if(populated_data.loc[i]['432571.0']==1):
        s = s+2


    weighted_charlson.append(s)
  


In [29]:
# Lets add our new weighted charlson feature to the populated data frame
# Then we filter the df to only include the person_id and charlson score

populated_data['weighted_charlson']=weighted_charlson
populated_data = populated_data.reset_index(drop=True)
populated_data = populated_data[['person_id','weighted_charlson']]
populated_data = populated_data.reset_index(drop=True)

In [30]:
# Preview of populated data

populated_data.head()

,person_id,weighted_charlson
0,138730,3
1,127727,4
2,46210,0
3,10249,0
4,78014,0


In [32]:
# We join the populated_data to our demographics so we can adjust charlson score by age

demographics = populated_data.join(demographics.set_index('person_id'), on='person_id')

In [35]:
# Preview demographics

demographics.head()

,person_id,weighted_charlson,year_of_birth,gender_source_value,ethnicity_source_value,death_datetime,latest_visit_date,age_last_visit,outcome
0,138730,3,1981.0,1,1,2009-10-02 11:50:35,2009-07-07 13:12:14,28.0,1
1,127727,4,1944.0,1,2,2009-07-25 23:38:38,2009-02-21 13:11:27,65.0,1
2,46210,0,1941.0,1,3,2010-08-03 19:47:22,2010-05-20 12:20:52,69.0,1
3,10249,0,1929.0,2,2,2010-05-26 23:14:58,2009-12-01 16:23:09,80.0,1
4,78014,0,1931.0,1,1,2010-07-23 23:37:41,2010-05-05 05:51:12,79.0,1


In [40]:
# Let's update Charlson index to include age
# We add the age score from the charlson index paper to the current score

updated = []
for i in range(len(demographics)):
    age_score = 0
    if(demographics.loc[i]['age_last_visit'] in range(50,60)):
        age_score = 1
    elif(demographics.loc[i]['age_last_visit'] in range(60,70)):
        age_score = 2
    elif(demographics.loc[i]['age_last_visit'] in range(70,80)):
        age_score = 3
    elif(demographics.loc[i]['age_last_visit'] >= 80):
        age_score = 4
  
    updated_score = age_score + demographics.loc[i]['weighted_charlson']
  
    updated.append(updated_score)

demographics['age_Charlson_index'] = updated

In [43]:
# Preview of updated demographics

demographics.head()

,person_id,weighted_charlson,year_of_birth,gender_source_value,ethnicity_source_value,death_datetime,latest_visit_date,age_last_visit,outcome,age_Charlson_index
0,138730,3,1981.0,1,1,2009-10-02 11:50:35,2009-07-07 13:12:14,28.0,1,3
1,127727,4,1944.0,1,2,2009-07-25 23:38:38,2009-02-21 13:11:27,65.0,1,6
2,46210,0,1941.0,1,3,2010-08-03 19:47:22,2010-05-20 12:20:52,69.0,1,2
3,10249,0,1929.0,2,2,2010-05-26 23:14:58,2009-12-01 16:23:09,80.0,1,4
4,78014,0,1931.0,1,1,2010-07-23 23:37:41,2010-05-05 05:51:12,79.0,1,3


In [46]:
# Lets explore the charlson index breakdown between two outcomes

demographics[['age_Charlson_index','outcome']].groupby('outcome').mean()

,age_Charlson_index
outcome,
0,3.492089
1,3.484615


# Comment

There is no difference between the charlson index score for my training set. My theory is because we are missing some very important concept id codes that were not found in the input data. 

# Export Data

In [47]:
# Lets filter our demographics to only include the features we need
# reset index

demographics = demographics[['person_id','outcome','age_Charlson_index','gender_source_value',
                             'ethnicity_source_value','age_last_visit']]
demographics = demographics.reset_index(drop=True)


demographics.to_csv('output/step_3_train_demographics.csv',index=False)


